In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

movies = pd.read_csv('/content/drive/MyDrive/머신러닝/data/Movie/tmdb_5000_movies.csv')
movies.shape

(4803, 20)

In [4]:
df_movies = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [5]:
df_movies.dropna(inplace=True)
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4800 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4800 non-null   int64  
 1   title         4800 non-null   object 
 2   genres        4800 non-null   object 
 3   vote_average  4800 non-null   float64
 4   vote_count    4800 non-null   int64  
 5   popularity    4800 non-null   float64
 6   keywords      4800 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 337.5+ KB


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [6]:
from ast import literal_eval

df_movies['genres']=df_movies['genres'].apply(literal_eval)
print(df_movies['genres'][:1][0][0])

{'id': 28, 'name': 'Action'}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
df_movies['genres'] = df_movies['genres'].apply(lambda x : [y['name'] for y in x])
df_movies['genres'][:2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0    [Action, Adventure, Fantasy, Science Fiction]
1                     [Adventure, Fantasy, Action]
Name: genres, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

df_movies['genres_literal'] = df_movies['genres'].apply(lambda x : (' ').join(x))
print(type(df_movies['genres_literal'][0]))
df_movies['genres_literal'][0]

countVect = CountVectorizer(ngram_range=(1,2)).fit(df_movies['genres_literal'])
countVect.vocabulary_

In [9]:
genre_matrix = countVect.fit_transform(df_movies['genres_literal'])
genre_matrix.shape

from sklearn.metrics.pairwise import cosine_similarity

genre_similarity = cosine_similarity(genre_matrix, genre_matrix)
genre_similarity[:3]


array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.        , 0.        ,
        0.        ],
       [0.4472136 , 0.4       , 1.        , ..., 0.        , 0.        ,
        0.        ]])

In [10]:
genre_similarity_sorted_idx = genre_similarity.argsort()[:,::-1]
genre_similarity_sorted_idx[:3]

array([[   0,   46,  813, ..., 3037, 3036, 2399],
       [ 329,  129,    1, ..., 3066, 3065, 2399],
       [1542,    2, 1740, ..., 2998, 2997, 2399]])

In [11]:
def find_similarity_movie(dataFrame, similarity_sorted_idx, movieName, top=10):
  title_movie = dataFrame[dataFrame['title'].str.lower().isin([movieName.lower()])]
  title_idx = title_movie.index.values
  similar_indexes = similarity_sorted_idx[title_idx, :top]
  print(similar_indexes.ndim, '차원 : ', similar_indexes)
  similar_indexes = similar_indexes.reshape(-1)
  print(similar_indexes.ndim, '차원 : ', similar_indexes)
  
  return dataFrame.iloc[similar_indexes]

find_similarity_movie(df_movies, genre_similarity_sorted_idx, 'avatar')

2 차원 :  [[   0   46  813  870   14 3493 1296 1652  419  420]]
1 차원 :  [   0   46  813  870   14 3493 1296 1652  419  420]


,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction
46,127585,X-Men: Days of Future Past,"[Action, Adventure, Fantasy, Science Fiction]",7.5,6032,118.078691,"[{""id"": 1228, ""name"": ""1970s""}, {""id"": 1852, ""...",The ultimate X-Men ensemble fights a war for t...,Action Adventure Fantasy Science Fiction
813,1924,Superman,"[Action, Adventure, Fantasy, Science Fiction]",6.9,1022,48.507081,"[{""id"": 83, ""name"": ""saving the world""}, {""id""...",Mild-mannered Clark Kent works as a reporter a...,Action Adventure Fantasy Science Fiction
870,8536,Superman II,"[Action, Adventure, Fantasy, Science Fiction]",6.5,629,30.515175,"[{""id"": 83, ""name"": ""saving the world""}, {""id""...",Three escaped criminals from the planet Krypto...,Action Adventure Fantasy Science Fiction
14,49521,Man of Steel,"[Action, Adventure, Fantasy, Science Fiction]",6.5,6359,99.398009,"[{""id"": 83, ""name"": ""saving the world""}, {""id""...",A young boy learns that he has extraordinary p...,Action Adventure Fantasy Science Fiction
3494,27549,Beastmaster 2: Through the Portal of Time,"[Action, Adventure, Fantasy, Science Fiction]",4.6,17,1.478505,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","Mark Singer returns as Dar, the warrior who ca...",Action Adventure Fantasy Science Fiction
1296,9531,Superman III,"[Comedy, Action, Adventure, Fantasy, Science F...",5.3,490,22.164202,"[{""id"": 83, ""name"": ""saving the world""}, {""id""...","Aiming to defeat the Man of Steel, wealthy exe...",Comedy Action Adventure Fantasy Science Fiction
1652,14164,Dragonball Evolution,"[Action, Adventure, Fantasy, Science Fiction, ...",2.9,462,21.677732,"[{""id"": 3436, ""name"": ""karate""}, {""id"": 9715, ...",The young warrior Son Goku sets out on a quest...,Action Adventure Fantasy Science Fiction Thriller
419,8247,Jumper,"[Adventure, Fantasy, Science Fiction]",5.9,1799,21.218000,"[{""id"": 704, ""name"": ""adolescence""}, {""id"": 81...","David Rice is a man who knows no boundaries, a...",Adventure Fantasy Science Fiction
420,11253,Hellboy II: The Golden Army,"[Adventure, Fantasy, Science Fiction]",6.5,1527,58.579760,"[{""id"": 2096, ""name"": ""auction""}, {""id"": 7005,...",In this continuation to the adventure of the d...,Adventure Fantasy Science Fiction


In [12]:
movies_top10 = find_similarity_movie(df_movies, genre_similarity_sorted_idx, 'avatar')
movies_top10[['title', 'vote_average']]

2 차원 :  [[   0   46  813  870   14 3493 1296 1652  419  420]]
1 차원 :  [   0   46  813  870   14 3493 1296 1652  419  420]


,title,vote_average
0,Avatar,7.2
46,X-Men: Days of Future Past,7.5
813,Superman,6.9
870,Superman II,6.5
14,Man of Steel,6.5
3494,Beastmaster 2: Through the Portal of Time,4.6
1296,Superman III,5.3
1652,Dragonball Evolution,2.9
419,Jumper,5.9
420,Hellboy II: The Golden Army,6.5


In [15]:
df_movies[['title', 'vote_average', 'vote_count']]
df_movies[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
4247,Me You and Five Bucks,10.0,2
4662,Little Big Top,10.0,1
4045,"Dancer, Texas Pop. 81",10.0,1
3519,Stiff Upper Lips,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
1881,The Shawshank Redemption,8.5,8205
2970,There Goes My Baby,8.5,2
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [16]:
# 전체 영화에 대한 평균 평점
df_movies['vote_average']
C = df_movies['vote_average'].mean()

In [21]:
# m : 평점을 부여하기 위한 최소 투표 횟수
m = df_movies['vote_count'].quantile(0.6)

In [18]:
# 가중 평점(Weighted Rating) = (v/v(v+m)) * R + (m/(v+m)) * C
# v : 개별 영화에 평점을 투표한 횟수
v = df_movies['vote_count']

In [19]:
# R : 개별 영화에 대한 평균 평점
R = df_movies['vote_average']

In [31]:
def get_weighted_vote_average(df_movies):
  v = df_movies['vote_count']
  m = df_movies['vote_count'].quantile(0.6)
  R = df_movies['vote_average']
  C = df_movies['vote_average'].mean()
  
  return (v/(v+m)) * R + (m/(v+m)) * C

In [32]:
df_movies['weighted_vote_average'] = get_weighted_vote_average(df_movies)
df_movies[['title', 'vote_average', 'vote_count', 'weighted_vote_average']][:3]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,title,vote_average,vote_count,weighted_vote_average
0,Avatar,7.2,11800,7.166254
1,Pirates of the Caribbean: At World's End,6.9,4500,6.838528
2,Spectre,6.3,4466,6.284117


In [36]:
def find_similarity_movie(dataFrame, similarity_sorted_idx, movieName, top=10):
  title_movie = dataFrame[dataFrame['title'].str.lower().isin([movieName.lower()])]
  title_idx = title_movie.index.values
  similar_indexes = similarity_sorted_idx[title_idx, :top+5]
  # print(similar_indexes.ndim, '차원 : ', similar_indexes)
  similar_indexes = similar_indexes.reshape(-1)
  # print(similar_indexes.ndim, '차원 : ', similar_indexes)
  
  similar_indexes = similar_indexes[similar_indexes!=title_idx]

  return dataFrame.iloc[similar_indexes].sort_values('weighted_vote_average', ascending=False)[:top]

find_movies = find_similarity_movie(df_movies, genre_similarity_sorted_idx, 'avatar')
lst = ['title', 'vote_average', 'vote_count', 'weighted_vote_average']
find_movies[lst]

,title,vote_average,vote_count,weighted_vote_average
46,X-Men: Days of Future Past,7.5,6032,7.418471
813,Superman,6.9,1022,6.685048
14,Man of Steel,6.5,6359,6.477559
420,Hellboy II: The Golden Army,6.5,1527,6.420428
870,Superman II,6.5,629,6.348972
3208,Star Wars: Clone Wars: Volume 1,8.0,27,6.222292
1191,Small Soldiers,6.2,511,6.154957
1932,Sheena,5.0,22,6.031736
3494,Beastmaster 2: Through the Portal of Time,4.6,17,6.027505
419,Jumper,5.9,1799,5.932983


In [37]:
movies = [
  ['user1', '연평대전', 5],
  ['user1', '7번방의 선물', 4],
  ['user1', '국제시장', 4],
  ['user2', '연평대전', 5],
  ['user2', '7번방의 선물', 3],
  ['user2', '국제시장', 4],
  ['user2', '명랑', 5],
  ['user2', '국가대표', 3],
  # ['user3', '연평대전', 4],
  ['user3', '7번방의 선물', 3],
  # ['user3', '국제시장', 3],
  ['user3', '명랑', 3],
  ['user3', '국가대표', 5]
]

In [46]:
df_movies = pd.DataFrame(movies, columns=['userID', 'movieName', 'Rating'])
df_movies

,userID,movieName,Rating
0,user1,연평대전,5
1,user1,7번방의 선물,4
2,user1,국제시장,4
3,user2,연평대전,5
4,user2,7번방의 선물,3
5,user2,국제시장,4
6,user2,명랑,5
7,user2,국가대표,3
8,user3,7번방의 선물,3
9,user3,명랑,3


In [47]:
# df_movies.pivot_table(데이터, row, col)
df_user_movies = df_movies.pivot_table('Rating', index='userID', columns='movieName')
df_user_movies

movieName,7번방의 선물,국가대표,국제시장,명랑,연평대전
userID,,,,,
user1,4.0,NaN,4.0,NaN,5.0
user2,3.0,3.0,4.0,5.0,5.0
user3,3.0,5.0,NaN,3.0,NaN


In [48]:
df_user_movies = df_user_movies.fillna(0)
df_user_movies

movieName,7번방의 선물,국가대표,국제시장,명랑,연평대전
userID,,,,,
user1,4.0,0.0,4.0,0.0,5.0
user2,3.0,3.0,4.0,5.0,5.0
user3,3.0,5.0,0.0,3.0,0.0


In [49]:
# 유클리디안 유사도
from sklearn.metrics.pairwise import euclidean_distances

euclidean_distances(df_user_movies, df_user_movies)

array([[0.        , 5.91607978, 8.71779789],
       [5.91607978, 0.        , 7.        ],
       [8.71779789, 7.        , 0.        ]])

In [51]:
# 코사인 유사도
from sklearn.metrics.pairwise import cosine_similarity

usr_sim = cosine_similarity(df_user_movies, df_user_movies)
usr_sim

array([[1.        , 0.76594714, 0.24238715],
       [0.76594714, 1.        , 0.64891937],
       [0.24238715, 0.64891937, 1.        ]])

In [52]:
usr_based_collab = pd.DataFrame(usr_sim, index=df_user_movies.index, columns=df_user_movies.index)
usr_based_collab

userID,user1,user2,user3
userID,,,
user1,1.000000,0.765947,0.242387
user2,0.765947,1.000000,0.648919
user3,0.242387,0.648919,1.000000


In [53]:
usr_based_collab['user1'].sort_values(ascending=False)

userID
user1    1.000000
user2    0.765947
user3    0.242387
Name: user1, dtype: float64

In [54]:
# user2가 본 영화 추천
df_user_movies

movieName,7번방의 선물,국가대표,국제시장,명랑,연평대전
userID,,,,,
user1,4.0,0.0,4.0,0.0,5.0
user2,3.0,3.0,4.0,5.0,5.0
user3,3.0,5.0,0.0,3.0,0.0


In [56]:
df_user_movies.loc['user2'].sort_values(ascending=False)

movieName
명랑         5.0
연평대전       5.0
국제시장       4.0
7번방의 선물    3.0
국가대표       3.0
Name: user2, dtype: float64

In [57]:
# 전치
df_user_movies_T = df_user_movies.transpose()

df_user_movies_T[df_user_movies_T['user1']==0]

userID,user1,user2,user3
movieName,,,
국가대표,0.0,3.0,5.0
명랑,0.0,5.0,3.0


In [60]:
df_user=df_user_movies_T[df_user_movies_T['user1']==0]
df_user['user2'].sort_values(ascending=False)

movieName
명랑      5.0
국가대표    3.0
Name: user2, dtype: float64

In [61]:
# 아이템 기반 추천 알고리즘
df_user_movies_T

userID,user1,user2,user3
movieName,,,
7번방의 선물,4.0,3.0,3.0
국가대표,0.0,3.0,5.0
국제시장,4.0,4.0,0.0
명랑,0.0,5.0,3.0
연평대전,5.0,5.0,0.0


In [62]:
item_based_sim = cosine_similarity(df_user_movies_T, df_user_movies_T)
item_based_sim

array([[1.        , 0.70588235, 0.84887469, 0.70588235, 0.84887469],
       [0.70588235, 1.        , 0.36380344, 0.88235294, 0.36380344],
       [0.84887469, 0.36380344, 1.        , 0.60633906, 1.        ],
       [0.70588235, 0.88235294, 0.60633906, 1.        , 0.60633906],
       [0.84887469, 0.36380344, 1.        , 0.60633906, 1.        ]])

In [63]:
# item_based_sim.dot(df_user_movies)
df_user_movies.dot(item_based_sim)

,0,1,2,3,4
userID,,,,,
user1,11.639872,6.097760,12.395499,8.280581,12.395499
user2,16.286931,12.803643,15.669730,15.221757,15.669730
user3,8.647059,9.764706,6.184658,9.529412,6.184658


In [64]:
np.abs(item_based_sim)
np.abs(item_based_sim).sum(axis=1)

array([4.10951408, 3.31584217, 3.81901719, 3.80091342, 3.81901719])

In [66]:
df_pred_movies=df_user_movies.dot(item_based_sim)/np.abs(item_based_sim).sum(axis=1)
df_pred_movies

,0,1,2,3,4
userID,,,,,
user1,2.832421,1.838978,3.245730,2.178577,3.245730
user2,3.963226,3.861355,4.103079,4.004763,4.103079
user3,2.104156,2.944864,1.619437,2.507137,1.619437


In [67]:
df_pred_movies.values-df_user_movies.values

array([[-1.16757944,  1.83897786, -0.75426998,  2.17857658, -1.75426998],
       [ 0.9632255 ,  0.86135469,  0.10307912, -0.995237  , -0.89692088],
       [-0.89584397, -2.0551355 ,  1.61943718, -0.49286271,  1.61943718]])

In [68]:
rating_square = np.square(df_pred_movies.values-df_user_movies.values)
rating_square

array([[1.36324174, 3.38183958, 0.5689232 , 4.74619593, 3.07746315],
       [0.92780337, 0.7419319 , 0.01062531, 0.99049668, 0.80446706],
       [0.80253642, 4.22358194, 2.62257679, 0.24291365, 2.62257679]])

In [69]:
rating_square.mean()

1.8084782338349181

In [70]:
%timeit rating_square.reshape(-1).mean()
%timeit rating_square.flatten().mean()
%timeit rating_square.ravel().mean()

The slowest run took 26.53 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 7.1 µs per loop
The slowest run took 9.86 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 7.35 µs per loop
The slowest run took 34.18 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 216 ns per loop


In [71]:
df_user_movies.values

array([[4., 0., 4., 0., 5.],
       [3., 3., 4., 5., 5.],
       [3., 5., 0., 3., 0.]])

In [72]:
df_user_movies.values.nonzero()

(array([0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2]),
 array([0, 2, 4, 0, 1, 2, 3, 4, 0, 1, 3]))

In [75]:
user_movies_nonzero = df_user_movies.values[df_user_movies.values.nonzero()]

In [76]:
pred_movies_nonzero = df_pred_movies.values[df_user_movies.values.nonzero()]

In [77]:
np.square(user_movies_nonzero-pred_movies_nonzero).mean()

1.250362218706176

In [79]:
ratings_arr=df_user_movies.values
ratings_arr

array([[4., 0., 4., 0., 5.],
       [3., 3., 4., 5., 5.],
       [3., 5., 0., 3., 0.]])

In [81]:
ratings_arr[0, :]

array([4., 0., 4., 0., 5.])

In [83]:
ratings_arg = np.argsort(ratings_arr[0, :])
ratings_arg 

array([1, 3, 0, 2, 4])

In [87]:
ratings_arg[:4]
ratings_arg[:-1]
ratings_arg[:-4]
ratings_arg[:-4:-1]

array([4, 2, 0])

In [89]:
n=3
top_n_arg = np.argsort(ratings_arr[0,:])[:-1-n:-1]

In [90]:
ratings_arr[0][top_n_arg]

array([5., 4., 4.])